In [2]:
import pandas as pd

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
dataset=pd.read_csv("vehicle_maintenance_data.csv")
print(dataset)

      Vehicle_Model  Mileage Maintenance_History  Reported_Issues  \
0             Truck    58765                Good                0   
1               Van    60353             Average                1   
2               Bus    68072                Poor                0   
3               Bus    60849             Average                4   
4               Bus    45742                Poor                5   
...             ...      ...                 ...              ...   
49995           Car    77229                Good                3   
49996         Truck    54742                Good                4   
49997           Van    75601                Poor                0   
49998           Car    66181                Good                1   
49999           Car    71605                Poor                0   

       Vehicle_Age Fuel_Type Transmission_Type  Engine_Size  Odometer_Reading  \
0                4  Electric         Automatic         2000             28524   
1        

In [5]:
dataset=pd.get_dummies(dataset,drop_first=True)
print(dataset)

       Mileage  Reported_Issues  Vehicle_Age  Engine_Size  Odometer_Reading  \
0        58765                0            4         2000             28524   
1        60353                1            7         2500            133630   
2        68072                0            2         1500             34022   
3        60849                4            5         2500             81636   
4        45742                5            1         2000             97162   
...        ...              ...          ...          ...               ...   
49995    77229                3            6         2500             97404   
49996    54742                4            4          800             13421   
49997    75601                0            4         1500            124851   
49998    66181                1            7         2500            103182   
49999    71605                0           10         2000             65721   

       Insurance_Premium  Service_History  Accident

In [6]:
dataset.columns

Index(['Mileage', 'Reported_Issues', 'Vehicle_Age', 'Engine_Size',
       'Odometer_Reading', 'Insurance_Premium', 'Service_History',
       'Accident_History', 'Fuel_Efficiency', 'Need_Maintenance',
       ...
       'Warranty_Expiry_Date_2026-03-28', 'Warranty_Expiry_Date_2026-03-29',
       'Owner_Type_Second', 'Owner_Type_Third', 'Tire_Condition_New',
       'Tire_Condition_Worn Out', 'Brake_Condition_New',
       'Brake_Condition_Worn Out', 'Battery_Status_New',
       'Battery_Status_Weak'],
      dtype='object', length=1063)

In [8]:
dataset["Need_Maintenance"].value_counts()

Need_Maintenance
1    40498
0     9502
Name: count, dtype: int64

In [9]:
independent=dataset[['Mileage', 'Reported_Issues', 'Vehicle_Age', 'Engine_Size',
       'Odometer_Reading', 'Insurance_Premium', 'Service_History',
       'Accident_History', 'Fuel_Efficiency', 
       'Owner_Type_Second', 'Owner_Type_Third', 'Tire_Condition_New',
       'Tire_Condition_Worn Out', 'Brake_Condition_New',
       'Brake_Condition_Worn Out', 'Battery_Status_New',
       'Battery_Status_Weak']]
print(independent)

       Mileage  Reported_Issues  Vehicle_Age  Engine_Size  Odometer_Reading  \
0        58765                0            4         2000             28524   
1        60353                1            7         2500            133630   
2        68072                0            2         1500             34022   
3        60849                4            5         2500             81636   
4        45742                5            1         2000             97162   
...        ...              ...          ...          ...               ...   
49995    77229                3            6         2500             97404   
49996    54742                4            4          800             13421   
49997    75601                0            4         1500            124851   
49998    66181                1            7         2500            103182   
49999    71605                0           10         2000             65721   

       Insurance_Premium  Service_History  Accident

In [10]:
dependent=dataset[["Need_Maintenance"]] 
print(dependent)

       Need_Maintenance
0                     1
1                     1
2                     1
3                     1
4                     1
...                 ...
49995                 1
49996                 1
49997                 1
49998                 1
49999                 0

[50000 rows x 1 columns]


In [11]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(independent, dependent, test_size=0.30,random_state=0)

In [12]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)
x_train

array([[ 1.16423527, -0.87415278,  1.21674369, ..., -0.71117152,
        -0.70744016, -0.70734923],
       [ 1.0965471 ,  1.46838859,  0.52137644, ..., -0.71117152,
        -0.70744016,  1.41372883],
       [ 1.02767994,  0.88275325, -0.86935808, ...,  1.40613055,
         1.41354712, -0.70734923],
       ...,
       [ 0.41293821, -0.28851743,  0.52137644, ...,  1.40613055,
         1.41354712, -0.70734923],
       [-0.09541657, -0.28851743,  1.21674369, ..., -0.71117152,
        -0.70744016,  1.41372883],
       [-0.10741458,  0.88275325,  1.21674369, ...,  1.40613055,
        -0.70744016, -0.70734923]], shape=(35000, 17))

In [13]:
from sklearn. ensemble import RandomForestClassifier

In [14]:
from sklearn.model_selection import GridSearchCV  


param_grid = {'criterion':['gini','entropy', 'log_loss'] }
grid=GridSearchCV(RandomForestClassifier(),param_grid ,refit=True,verbose=3,n_jobs=-1,scoring="f1_weighted")
grid.fit(x_train,y_train)


Fitting 5 folds for each of 3 candidates, totalling 15 fits


,estimator,RandomForestClassifier()
,param_grid,"{'criterion': ['gini', 'entropy', ...]}"
,scoring,'f1_weighted'
,n_jobs,-1
,refit,True
,cv,None
,verbose,3
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_estimators,100


In [15]:
re=grid.cv_results_
grid_predictions=grid.predict(x_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)


from sklearn.metrics import classification_report
report=classification_report(y_test,grid_predictions)


In [16]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("the f1_macro value for best parameter{}:".format(grid.best_params_),f1_macro)


the f1_macro value for best parameter{'criterion': 'gini'}: 0.9549499707735192


In [17]:
print("The confusion Matrix:/n",cm)

The confusion Matrix:/n [[ 2518   367]
 [  306 11809]]


In [18]:
print("The confusion Matrix:/n",cm)

The confusion Matrix:/n [[ 2518   367]
 [  306 11809]]


In [19]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9907120167716804

In [20]:
table=pd.DataFrame.from_dict(re)
print(table)

   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       9.656724      0.217138         0.231485        0.005903   
1      11.646302      0.410781         0.210820        0.009236   
2      11.113855      0.187632         0.191279        0.014945   

  param_criterion                     params  split0_test_score  \
0            gini      {'criterion': 'gini'}           0.960230   
1         entropy   {'criterion': 'entropy'}           0.959201   
2        log_loss  {'criterion': 'log_loss'}           0.959739   

   split1_test_score  split2_test_score  split3_test_score  split4_test_score  \
0           0.956857           0.957365           0.956721           0.961095   
1           0.954967           0.957865           0.956052           0.959466   
2           0.956311           0.957180           0.955335           0.959193   

   mean_test_score  std_test_score  rank_test_score  
0         0.958454        0.001837                1  
1         0.957510        0.0

In [24]:
vehicle_age_input= float(input("vehicle_Age:"))
Reported_Issues_input=float(input("Reported_Issues:"))
Mileage_input=float(input("Mileage:"))
Tire_Condition_New_input=int(input("Tire_Condition 0 or 1:"))
Need_Maintenance_input=int(input("Maintenance 0 or 1:"))

vehicle_Age: 12
Reported_Issues: 3
Mileage: 3456
Tire_Condition 0 or 1: 0
Maintenance 0 or 1: 1


In [38]:
Future_Prediction=grid.predict([[independent]])
print("Future_Prediction={}".format(Future_Prediction))

ValueError: Found array with dim 4, while dim <= 2 is required by RandomForestClassifier.